# MODELO DE APRENDIZAJE BERT PARA PREDICCIÓN DE NEUROPÉPTIDOS

## INSTALACIÓN E IMPORTACIÓN DE PAQUETES DE TRABAJO

In [ ]:
!pip install transformers

In [ ]:
import os
import pandas as pd
import numpy as np
import torch
import re
from pprint import pprint
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from torch.utils.data import Dataset, DataLoader

torch.cuda.is_available()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

from transformers import AutoTokenizer, Trainer, TrainingArguments, BertForSequenceClassification, AdamW

cuda


In [ ]:
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'  # or any {'0', '1', '2'}
import tensorflow as tf

## FUNCIÓN TOKENIZACIÓN

In [ ]:
# Clase mix_data para procesar cualquier DataFrame (df o ddf)
class mix_data():
    def __init__(self, df, tokenizer_name='Rostlab/prot_bert_bfd', max_len=200):
        # Inicializa el tokenizador de BERT y otros parámetros
        self.tokenizer = AutoTokenizer.from_pretrained(tokenizer_name, do_lower_case=False)
        self.max_len = max_len
        self.seqs, self.labels = self.get_seqs_labels(df)  # Aquí pasamos df o ddf como parámetro

    def get_seqs_labels(self, df):  # Aquí recibimos el DataFrame que puede ser df o ddf
        # Aislar las secuencias de aminoácidos y sus etiquetas
        seqs = list(df['aa_seq'])  # Suponiendo que 'aa_seq' tiene las secuencias
        labels = list(df['label'].astype(int))  # Suponiendo que 'label' tiene las etiquetas
        return seqs, labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        seq = " ".join("".join(self.seqs[idx].split()))  # Tokeniza la secuencia
        seq_ids = self.tokenizer(seq, truncation=True, padding='max_length', max_length=self.max_len)

        sample = {key: torch.tensor(val) for key, val in seq_ids.items()}
        sample['labels'] = torch.tensor(self.labels[idx])  # Añadir la etiqueta correspondiente
        return sample
    def __setitem__(self, idx, sample):
        # Update the elements of the mix_data object at the given index
        for key in sample:
            if key in self.__dict__:  # Or any other logic to determine which attributes to update
                if isinstance(self.__dict__[key], list): # Assuming your data is stored in lists
                    self.__dict__[key][idx] = sample[key]
                # Add other conditions for different data structures if needed
                else:
                    # Handle cases where the attribute is not a list
                    pass  # Or raise an error if this should not happen


## CARGADO DE DATOS

In [ ]:
# read in the train dataset
# create the dataset

data_url = 'https://raw.githubusercontent.com/JavierColubi/Javier-Colubi-github/refs/heads/main/train_neuropep-BERT_2.0.txt'
df = pd.read_csv(data_url, index_col = 0)
df = df.sample(frac=1, random_state = 0)
print(df.head(7))
print(f'El número de secuencias totales en el archivo es: {len(df)}')

train_dataset = mix_data(df)

                                                            aa_seq  aa_len  \
non_AMPEP63746                               MAKINELLRESTTTNSNSIGR      21   
IDEN1606         LYLKQADFDDPRMFTSSFGKRSAIESEPQAYPKSYRAIRIQRRSMD...      99   
IDEN1310                                                  KVKFSAWG       8   
UniRef50_Q9ZTK5                  CVEKTFIFPPEAIEKLKSKAVEFGIEKPTRVEV      33   
IDEN135                       APLEPMYPGDYATPEQMAQYETQLRRYINTLTRPRY      36   
IDEN265                                                  DGRMYSFGL       9   
non_AMPEP34652                            THPSVLPFIKQLIGTMDSVRGLPR      24   

                 label  
non_AMPEP63746       0  
IDEN1606             2  
IDEN1310             2  
UniRef50_Q9ZTK5      0  
IDEN135              2  
IDEN265              2  
non_AMPEP34652       0  
El número de secuencias totales en el archivo es: 6266


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/86.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/361 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/81.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [ ]:
# read in the train dataset
# create the dataset

data_url2 = 'https://raw.githubusercontent.com/JavierColubi/Javier-Colubi-github/refs/heads/main/validacion_20.txt'
ddf = pd.read_csv(data_url2, index_col = 0)
ddf = ddf.sample(frac=1, random_state = 0)
print(ddf.head(7))
print(f'El número de secuencias totales en el archivo es: {len(ddf)}')

val_dataset = mix_data(ddf)

                                                            aa_seq  aa_len  \
DRAMP04081                                     AKKVFKRLEKLFSKIFNFK      19   
IDEN2055                                           QPPYLDLTPAYFHIR      15   
non_AMPEP52184                                     AMTEELEAVENNGVR      15   
non_AMPEP149113                                      PFDRISNSAFSDF      13   
IDEN1925                   LVMAGYERRGIQKRHGEQGITCECCYNHCSFRELVQYCN      39   
IDEN2021                     EAVPSQVLSEQNEEAGAPLSPLSEMPPWMGEVNPAQR      37   
DRAMP00144       NRWTNAYSAALGCAVPGVKYGKKLGGVWGAVIGGVGGAAVCGLAGY...      50   

                 label  
DRAMP04081           1  
IDEN2055             2  
non_AMPEP52184       0  
non_AMPEP149113      0  
IDEN1925             2  
IDEN2021             2  
DRAMP00144           1  
El número de secuencias totales en el archivo es: 1065


## FUNCIÓN DE EVALUCIÓN DEL ENTRENAMIENTO

In [ ]:
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)

    # Calcular métricas globales (macro)
    precision_macro, recall_macro, f1_macro, _ = precision_recall_fscore_support(labels, preds, average='weighted')
    acc = accuracy_score(labels, preds)

    # Calcular métricas por clase
    precision_per_label, recall_per_label, f1_per_label, support_per_label = precision_recall_fscore_support(
        labels, preds, average=None
    )

    # Confusion matrix
    conf_matrix = confusion_matrix(labels, preds).tolist()  # Convertir a lista para serialización en JSON si es necesario

    # Crear el diccionario de métricas
    metrics = {
        'accuracy': acc,
        'precision_macro': precision_macro,
        'recall_macro': recall_macro,
        'f1_macro': f1_macro,
    }

    # Agregar métricas por clase al diccionario
    for i, precision in enumerate(precision_per_label):
        metrics[f'precision_label_{i}'] = precision  # Unique key for each label's precision
    for i, recall in enumerate(recall_per_label):
        metrics[f'recall_label_{i}'] = recall  # Unique key for each label's recall
    for i, f1 in enumerate(f1_per_label):
        metrics[f'f1_label_{i}'] = f1  # Unique key for each label's F1-score
    # ... (add other metrics as needed) ...

    return metrics

## MODELO DE ENTRENAMIENTO

In [ ]:
# define the model initializing function for Trainer in huggingface

def model_init():
    model = BertForSequenceClassification.from_pretrained('Rostlab/prot_bert_bfd', num_labels=3)
    #dropout_rate = 0.5  # Cambia el valor según lo que quieras (0.1 a 0.5 suele ser común)
    #model.bert.encoder.config.hidden_dropout_prob = dropout_rate  # Dropout en las capas internas
    #model.bert.encoder.config.attention_probs_dropout_prob = dropout_rate  # Dropout en la atención
    #model.dropout = torch.nn.Dropout(p=dropout_rate)  # Dropout en la capa final (si está definida)
    return model

In [ ]:
# training on entire data
# no evaluation/validation

training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=10,
    learning_rate = 5e-5,
    per_device_train_batch_size=1,
    warmup_steps=0,
    weight_decay=0.1,
    logging_dir='./logs',
    do_train=True,
    do_eval=True,
    evaluation_strategy="epoch",
    save_strategy='epoch',
    logging_strategy='epoch',
    gradient_accumulation_steps=64,
    fp16=True,
    fp16_opt_level="O2",
    run_name="neuropep_BERT",
    seed=0,
    load_best_model_at_end = True,
    metric_for_best_model="eval_accuracy",  # Seleccionar el mejor modelo basado en precisión
    greater_is_better=True        # La precisión es mejor si es mayor
)

trainer = Trainer(
    model_init=model_init,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics = compute_metrics,
)
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/1.68G [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at Rostlab/prot_bert_bfd and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at Rostlab/prot_bert_bfd and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Epoch,Training Loss,Validation Loss,Accuracy,Precision Macro,Recall Macro,F1 Macro,Precision Label 0,Precision Label 1,Precision Label 2,Recall Label 0,Recall Label 1,Recall Label 2,F1 Label 0,F1 Label 1,F1 Label 2
0,0.846400,1.051599,0.520188,0.524410,0.520188,0.521466,0.369509,0.665746,0.537975,0.402817,0.678873,0.478873,0.385445,0.672245,0.506706
1,0.624800,1.031513,0.584038,0.566439,0.584038,0.564214,0.429864,0.713896,0.555556,0.267606,0.738028,0.746479,0.329861,0.725762,0.637019
2,0.526000,1.154910,0.568075,0.574182,0.568075,0.567723,0.408669,0.773463,0.540416,0.371831,0.673239,0.659155,0.389381,0.719880,0.593909
3,0.446100,1.015441,0.610329,0.609560,0.610329,0.605642,0.471761,0.763240,0.593679,0.400000,0.690141,0.740845,0.432927,0.724852,0.659148
4,0.369300,1.304759,0.590610,0.583127,0.590610,0.575300,0.434426,0.749216,0.565737,0.298592,0.673239,0.800000,0.353923,0.709199,0.662777
5,0.289700,1.304990,0.619718,0.615037,0.619718,0.615362,0.466238,0.756598,0.622276,0.408451,0.726761,0.723944,0.435435,0.741379,0.669271
6,0.236000,1.459364,0.606573,0.595702,0.606573,0.597202,0.456835,0.729805,0.600467,0.357746,0.738028,0.723944,0.401264,0.733894,0.656450
7,0.194000,1.474323,0.630047,0.618560,0.630047,0.612738,0.506494,0.742938,0.606250,0.329577,0.740845,0.819718,0.399317,0.741890,0.697006
8,0.169500,1.558082,0.620657,0.609900,0.620657,0.610528,0.485294,0.727778,0.616628,0.371831,0.738028,0.752113,0.421053,0.732867,0.677665
9,0.146800,1.620241,0.624413,0.611995,0.624413,0.613578,0.494424,0.711230,0.630332,0.374648,0.749296,0.749296,0.426282,0.729767,0.684685


TrainOutput(global_step=970, training_loss=0.38658978275417055, metrics={'train_runtime': 6417.909, 'train_samples_per_second': 9.763, 'train_steps_per_second': 0.151, 'total_flos': 2.8229621737728e+16, 'train_loss': 0.38658978275417055, 'epoch': 9.90743696137887})

## MÉTRICAS DEL MODELO

In [ ]:
# Imprimir el historial de logs del entrenador
print(trainer.state.log_history)

# Analizar la relación entre pasos y épocas
for log in trainer.state.log_history:
    if 'epoch' in log:
        print(f"Step: {log['step']}, Epoch: {log['epoch']}, Metrics: {log}")

[{'loss': 0.8464, 'grad_norm': 239.1932373046875, 'learning_rate': 4.5463917525773195e-05, 'epoch': 0.990743696137887, 'step': 97}, {'eval_loss': 1.0515987873077393, 'eval_accuracy': 0.5201877934272301, 'eval_precision_macro': 0.5244098612751812, 'eval_recall_macro': 0.5201877934272301, 'eval_f1_macro': 0.521465539835203, 'eval_precision_label_0': 0.3695090439276486, 'eval_precision_label_1': 0.6657458563535912, 'eval_precision_label_2': 0.5379746835443038, 'eval_recall_label_0': 0.4028169014084507, 'eval_recall_label_1': 0.6788732394366197, 'eval_recall_label_2': 0.4788732394366197, 'eval_f1_label_0': 0.38544474393531, 'eval_f1_label_1': 0.6722454672245467, 'eval_f1_label_2': 0.5067064083457526, 'eval_runtime': 12.7988, 'eval_samples_per_second': 83.211, 'eval_steps_per_second': 10.47, 'epoch': 0.990743696137887, 'step': 97}, {'loss': 0.6248, 'grad_norm': 3447.231689453125, 'learning_rate': 4.041237113402062e-05, 'epoch': 1.991701244813278, 'step': 195}, {'eval_loss': 1.03151321411132

In [ ]:
ls ./results

checkpoint-195/  checkpoint-391/  checkpoint-587/  checkpoint-783/  checkpoint-97/
checkpoint-293/  checkpoint-489/  checkpoint-685/  checkpoint-881/  checkpoint-970/


In [ ]:
#Este script permite elegir que te muestre las metricas de entrenamiento del epoch que quieras que en este
#caso será aquel que haya dado un valor de accuracy mayor en la validación, en este caso es el epoch 7
from transformers import AutoModelForSequenceClassification

# Ruta del checkpoint del epoch deseado
specific_epoch_checkpoint = "./results/checkpoint-783"  # Cambia X por el número del checkpoint

# Cargar el modelo desde el checkpoint
# Add the 'local_files_only=True' argument to load from local path.
model = AutoModelForSequenceClassification.from_pretrained(specific_epoch_checkpoint, local_files_only=True)
# Move the model to the GPU
model.to(device) # where 'device' is your cuda device if available
trainer.model = model  # Actualiza el modelo del trainer

# Calcular métricas para el conjunto de entrenamiento
train_predictions, train_label_ids, train_metrics = trainer.predict(train_dataset)
print(f"Metrics for epoch X:", train_metrics)

Metrics for epoch X: {'test_loss': 0.1606423258781433, 'test_accuracy': 0.9549952122566231, 'test_precision_macro': 0.9571567616403104, 'test_recall_macro': 0.9549952122566231, 'test_f1_macro': 0.9551918111741575, 'test_precision_label_0': 0.9893533123028391, 'test_precision_label_1': 0.9006243496357961, 'test_precision_label_2': 0.9646017699115044, 'test_recall_label_0': 0.9254887495389156, 'test_recall_label_1': 0.9735658042744657, 'test_recall_label_2': 0.9814293753517164, 'test_f1_label_0': 0.9563560129597866, 'test_f1_label_1': 0.9356756756756757, 'test_f1_label_2': 0.9729428172942818, 'test_runtime': 319.302, 'test_samples_per_second': 19.624, 'test_steps_per_second': 2.455}


In [ ]:
val_predictions, val_label_ids, val_metrics = trainer.predict(val_dataset)
print("Validation Metrics:", val_metrics)

Validation Metrics: {'test_loss': 1.4744986295700073, 'test_accuracy': 0.6300469483568075, 'test_precision_macro': 0.6188743700902875, 'test_recall_macro': 0.6300469483568075, 'test_f1_macro': 0.6126874115664851, 'test_precision_label_0': 0.508695652173913, 'test_precision_label_1': 0.7429378531073446, 'test_precision_label_2': 0.604989604989605, 'test_recall_label_0': 0.3295774647887324, 'test_recall_label_1': 0.7408450704225352, 'test_recall_label_2': 0.819718309859155, 'test_f1_label_0': 0.4, 'test_f1_label_1': 0.7418899858956276, 'test_f1_label_2': 0.6961722488038278, 'test_runtime': 55.6909, 'test_samples_per_second': 19.123, 'test_steps_per_second': 2.406}


## GUARDADO Y CARGADO DEL MODELO

In [ ]:
# save the model, if desired

from google.colab import drive
drive.mount('/content/drive')
# Call .contiguous() on the model's parameters before saving
for name, param in trainer.model.named_parameters():
    if not param.is_contiguous():
        param.data = param.data.contiguous()
trainer.save_model('/content/drive/MyDrive/Datos Finales para Memoria/modelos/neuropep-BERT/Fine-tuned_model/')

Mounted at /content/drive


In [ ]:
# predict AMP/non-AMP for a single example

# IMPORTANT:
# one must mount their Google Drive and load their own fine-tuned model before running the below cell for individual predictions
from google.colab import drive
drive.mount('/content/drive')

# load appropriate tokenizer and fine-tuned model
tokenizer = AutoTokenizer.from_pretrained('Rostlab/prot_bert_bfd', do_lower_case=False)
model = BertForSequenceClassification.from_pretrained("/content/drive/MyDrive/Datos Finales para Memoria/modelos/neuropep-BERT/Fine-tuned_model")

Mounted at /content/drive


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/86.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/361 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/81.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

## TESTS DE VERIFICACIÓN DEL ENTRENAMIENTO DEL MODELO

### TEST DE VERIFICACIÓN CON NEUROPÉPTIDOS HUMANOS

In [ ]:
import re
import torch

# Función para leer un archivo FASTA
def read_fasta(file_path):
    sequences = []
    with open(file_path, 'r') as f:
        identifier = ""
        sequence = ""
        for line in f:
            line = line.strip()
            if line.startswith(">"):  # Es un identificador de secuencia
                if sequence:  # Guarda la secuencia anterior si existe
                    sequences.append((identifier, sequence))
                    sequence = ""
                identifier = line[1:]  # Guarda el identificador sin el ">"
            else:
                sequence += line  # Agrega las líneas de la secuencia
        if sequence:  # Guarda la última secuencia
            sequences.append((identifier, sequence))
    return sequences
# Función para hacer predicciones de AMP/non-AMP para múltiples secuencias y guardar probabilidades
def predict_amp(sequences, output_file):
    i=0
    n=0
    with open(output_file, 'w') as out_f:
        out_f.write("Identifier\tSequence\tPrediction\tProbability\n")  # Encabezados

        for identifier, input_seq in sequences:
            input_seq_spaced = ' '.join([input_seq[i:i+1] for i in range(0, len(input_seq), 1)])
            input_seq_spaced = re.sub(r'[UZOB]', 'X', input_seq_spaced)  # Reemplazar aminoácidos no estándar
            input_seq_tok = tokenizer(input_seq_spaced, return_tensors='pt')
            #print(input_seq_tok)

            output = model(**input_seq_tok)
            logits = output[0]
           # print(f"Logits for {identifier}: {logits}")

            # Extraer la probabilidad de clase AMP
            #y_prob = torch.sigmoid(logits)[:, 1].detach().numpy()
            #y_pred = y_prob > 0.84  # Predicción binaria con umbral de 0.5
            #prob = y_prob[0]  # Probabilidad real (sin umbral)

            # Aplicar softmax para obtener probabilidades de cada clase
            y_prob = torch.softmax(logits, dim=1).detach().numpy()

            # Obtener la clase con la probabilidad más alta
            y_pred_class = y_prob.argmax(axis=1)[0]
            prob_class1, prob_class2, prob_class3 = y_prob[0]  # Probabilidades para las tres clases
            # Determinar la etiqueta de la predicción según la clase
            if y_pred_class == 0:
                input_class = 'non-AMP/OTHERS'  # Cambia estos nombres según tus etiquetas
                n=n+1
            elif y_pred_class == 1:
                input_class = 'AMP'
                n=n+1
            else:
                input_class = 'Neuropeptide'
                i=i+1
            # Escribir el identificador, secuencia, predicción y probabilidades en el archivo de salida
            out_f.write(f"{identifier}\t{input_seq}\t{input_class}\t{prob_class1:.4f}\t{prob_class2:.4f}\t{prob_class3:.4f}\n")
            print(f"Processed {identifier}: {input_class} (Probabilities: {prob_class1:.4f}, {prob_class2:.4f}, {prob_class3:.4f})")
    r=i/(i+n)
    print(f"El número de secuencias de Neuropeptidos es: {i}")
    print(f"El número de secuencias de AMP y non-AMP es: {n}")
    print(f"El ratio de acierto de neuropéptidos es de: {r}")

            # Determinar clase (AMP o non-AMP)
            #if y_pred:
            #    input_class = 'AMP'
            #else:
            #    input_class = 'non-AMP'

            # Escribir el identificador, secuencia, predicción y probabilidad en el archivo de salida
           # out_f.write(f"{identifier}\t{input_seq}\t{input_class}\t{prob:.4f}\n")
           # print(f"Processed {identifier}: {input_class} (Probability: {prob:.4f})")

# Ruta del archivo FASTA de entrada y del archivo de salida
fasta_file = '/content/drive/MyDrive/data_final/Human_neuropep_Neuropedia_test_NO_REPES.fasta'  # Nombre del archivo FASTA con secuencias
output_file = '/content/drive/MyDrive/Datos Finales para Memoria/resultados/RESULTADO_verificacion_neuropep_humanos.tsv'  # Archivo de salida con predicciones y probabilidades

# Leer las secuencias del archivo FASTA
sequences = read_fasta(fasta_file)

# Realizar predicciones y escribir resultados en un archivo
predict_amp(sequences, output_file)


Processed Homo_sapiens_(Human)_CRH-related_gene_family_Urotensin-2B: AMP (Probabilities: 0.0856, 0.9015, 0.0129)
Processed Homo_sapiens_(Human)_Somatostatin_gene_family_Somatostatin-14: AMP (Probabilities: 0.0794, 0.9075, 0.0131)
Processed Homo_sapiens_(Human)_Neurexophilins_Neurexophilin-1: AMP (Probabilities: 0.0712, 0.9152, 0.0137)
Processed Homo_sapiens_(Human)_Galanin_family_Galanin-like_peptide: Neuropeptide (Probabilities: 0.0077, 0.0091, 0.9832)
Processed Homo_sapiens_(Human)_Bombesin-like_peptide_gene_family_Neuromedin-B-32: Neuropeptide (Probabilities: 0.0274, 0.0168, 0.9558)
Processed Homo_sapiens_(Human)_Granins_Neurosecretory_protein_VGF: Neuropeptide (Probabilities: 0.0491, 0.0187, 0.9322)
Processed Homo_sapiens_(Human)_No-family_neuropeptides_Agouti-related_protein: Neuropeptide (Probabilities: 0.2055, 0.2946, 0.4999)
Processed Homo_sapiens_(Human)_Calcitonin_gene_family_Proadrenomedullin_N-20_terminal_peptide: Neuropeptide (Probabilities: 0.0246, 0.0304, 0.9451)
Process

### TEST DE VERIFICACIÓN CON AMPs DIFERENTES A LOS DEL ENTRENAMIENTO

In [ ]:
import re
import torch

# Función para leer un archivo FASTA
def read_fasta(file_path):
    sequences = []
    with open(file_path, 'r') as f:
        identifier = ""
        sequence = ""
        for line in f:
            line = line.strip()
            if line.startswith(">"):  # Es un identificador de secuencia
                if sequence:  # Guarda la secuencia anterior si existe
                    sequences.append((identifier, sequence))
                    sequence = ""
                identifier = line[1:]  # Guarda el identificador sin el ">"
            else:
                sequence += line  # Agrega las líneas de la secuencia
        if sequence:  # Guarda la última secuencia
            sequences.append((identifier, sequence))
    return sequences
# Función para hacer predicciones de AMP/non-AMP para múltiples secuencias y guardar probabilidades
def predict_amp(sequences, output_file):
    i=0
    n=0
    with open(output_file, 'w') as out_f:
        out_f.write("Identifier\tSequence\tPrediction\tProbability\n")  # Encabezados

        for identifier, input_seq in sequences:
            input_seq_spaced = ' '.join([input_seq[i:i+1] for i in range(0, len(input_seq), 1)])
            input_seq_spaced = re.sub(r'[UZOB]', 'X', input_seq_spaced)  # Reemplazar aminoácidos no estándar
            input_seq_tok = tokenizer(input_seq_spaced, return_tensors='pt')
            #print(input_seq_tok)

            output = model(**input_seq_tok)
            logits = output[0]
           # print(f"Logits for {identifier}: {logits}")

            # Extraer la probabilidad de clase AMP
            #y_prob = torch.sigmoid(logits)[:, 1].detach().numpy()
            #y_pred = y_prob > 0.84  # Predicción binaria con umbral de 0.5
            #prob = y_prob[0]  # Probabilidad real (sin umbral)

            # Aplicar softmax para obtener probabilidades de cada clase
            y_prob = torch.softmax(logits, dim=1).detach().numpy()

            # Obtener la clase con la probabilidad más alta
            y_pred_class = y_prob.argmax(axis=1)[0]
            prob_class1, prob_class2, prob_class3 = y_prob[0]  # Probabilidades para las tres clases
            # Determinar la etiqueta de la predicción según la clase
            if y_pred_class == 0:
                input_class = 'non-AMP/OTHERS'  # Cambia estos nombres según tus etiquetas
                n=n+1
            elif y_pred_class == 1:
                input_class = 'AMP'
                i=i+1
            else:
                input_class = 'Neuropeptide'
                n=n+1
            # Escribir el identificador, secuencia, predicción y probabilidades en el archivo de salida
            out_f.write(f"{identifier}\t{input_seq}\t{input_class}\t{prob_class1:.4f}\t{prob_class2:.4f}\t{prob_class3:.4f}\n")
            print(f"Processed {identifier}: {input_class} (Probabilities: {prob_class1:.4f}, {prob_class2:.4f}, {prob_class3:.4f})")
    print(f"El número de secuencias de AMPs es: {i}")
    print(f"El número de secuencias de neuropéptidos y non-AMP es: {n}")

            # Determinar clase (AMP o non-AMP)
            #if y_pred:
            #    input_class = 'AMP'
            #else:
            #    input_class = 'non-AMP'

            # Escribir el identificador, secuencia, predicción y probabilidad en el archivo de salida
           # out_f.write(f"{identifier}\t{input_seq}\t{input_class}\t{prob:.4f}\n")
           # print(f"Processed {identifier}: {input_class} (Probability: {prob:.4f})")

# Ruta del archivo FASTA de entrada y del archivo de salida
fasta_file = '/content/drive/MyDrive/data_final/general_amps_DRAMP_cdhit_80_NO_REPES_recortado1892seq.fasta'  # Nombre del archivo FASTA con secuencias
output_file = '/content/drive/MyDrive/Datos Finales para Memoria/resultados_verificacion_general_AMPs.tsv'  # Archivo de salida con predicciones y probabilidades

# Leer las secuencias del archivo FASTA
sequences = read_fasta(fasta_file)

# Realizar predicciones y escribir resultados en un archivo
predict_amp(sequences, output_file)


Processed DRAMP00032: AMP (Probabilities: 0.0667, 0.9188, 0.0145)
Processed DRAMP00089: AMP (Probabilities: 0.0658, 0.9180, 0.0162)
Processed DRAMP00106: AMP (Probabilities: 0.0718, 0.9145, 0.0138)
Processed DRAMP00127: AMP (Probabilities: 0.0966, 0.8812, 0.0222)
Processed DRAMP00129: AMP (Probabilities: 0.0670, 0.9182, 0.0148)
Processed DRAMP00189: AMP (Probabilities: 0.0700, 0.9155, 0.0145)
Processed DRAMP00190: non-AMP/OTHERS (Probabilities: 0.5189, 0.4695, 0.0116)
Processed DRAMP00191: non-AMP/OTHERS (Probabilities: 0.8212, 0.0918, 0.0871)
Processed DRAMP00204: AMP (Probabilities: 0.1470, 0.7889, 0.0641)
Processed DRAMP00254: AMP (Probabilities: 0.0963, 0.8922, 0.0115)
Processed DRAMP00384: non-AMP/OTHERS (Probabilities: 0.6807, 0.0819, 0.2373)
Processed DRAMP00425: non-AMP/OTHERS (Probabilities: 0.9761, 0.0151, 0.0088)
Processed DRAMP00437: AMP (Probabilities: 0.0679, 0.9165, 0.0156)
Processed DRAMP00454: AMP (Probabilities: 0.0747, 0.9116, 0.0138)
Processed DRAMP00766: Neuropepti

### TEST DE VERIFICACIÓN CON NON-AMPs DIFERENTES A LOS DEL ENTRENAMIENTO

In [ ]:
import re
import torch

# Función para leer un archivo FASTA
def read_fasta(file_path):
    sequences = []
    with open(file_path, 'r') as f:
        identifier = ""
        sequence = ""
        for line in f:
            line = line.strip()
            if line.startswith(">"):  # Es un identificador de secuencia
                if sequence:  # Guarda la secuencia anterior si existe
                    sequences.append((identifier, sequence))
                    sequence = ""
                identifier = line[1:]  # Guarda el identificador sin el ">"
            else:
                sequence += line  # Agrega las líneas de la secuencia
        if sequence:  # Guarda la última secuencia
            sequences.append((identifier, sequence))
    return sequences
# Función para hacer predicciones de AMP/non-AMP para múltiples secuencias y guardar probabilidades
def predict_amp(sequences, output_file):
    i=0
    n=0
    with open(output_file, 'w') as out_f:
        out_f.write("Identifier\tSequence\tPrediction\tProbability\n")  # Encabezados

        for identifier, input_seq in sequences:
            input_seq_spaced = ' '.join([input_seq[i:i+1] for i in range(0, len(input_seq), 1)])
            input_seq_spaced = re.sub(r'[UZOB]', 'X', input_seq_spaced)  # Reemplazar aminoácidos no estándar
            input_seq_tok = tokenizer(input_seq_spaced, return_tensors='pt')
            #print(input_seq_tok)

            output = model(**input_seq_tok)
            logits = output[0]
           # print(f"Logits for {identifier}: {logits}")

            # Extraer la probabilidad de clase AMP
            #y_prob = torch.sigmoid(logits)[:, 1].detach().numpy()
            #y_pred = y_prob > 0.84  # Predicción binaria con umbral de 0.5
            #prob = y_prob[0]  # Probabilidad real (sin umbral)

            # Aplicar softmax para obtener probabilidades de cada clase
            y_prob = torch.softmax(logits, dim=1).detach().numpy()

            # Obtener la clase con la probabilidad más alta
            y_pred_class = y_prob.argmax(axis=1)[0]
            prob_class1, prob_class2, prob_class3 = y_prob[0]  # Probabilidades para las tres clases
            # Determinar la etiqueta de la predicción según la clase
            if y_pred_class == 0:
                input_class = 'non-AMP/OTHERS'  # Cambia estos nombres según tus etiquetas
                i=i+1
            elif y_pred_class == 1:
                input_class = 'AMP'
                n=n+1
            else:
                input_class = 'Neuropeptide'
                n=n+1
            # Escribir el identificador, secuencia, predicción y probabilidades en el archivo de salida
            out_f.write(f"{identifier}\t{input_seq}\t{input_class}\t{prob_class1:.4f}\t{prob_class2:.4f}\t{prob_class3:.4f}\n")
            print(f"Processed {identifier}: {input_class} (Probabilities: {prob_class1:.4f}, {prob_class2:.4f}, {prob_class3:.4f})")
    print(f"El número de secuencias de Non-AMPs es: {i}")
    print(f"El número de secuencias de AMP y neuropéptidos es: {n}")

            # Determinar clase (AMP o non-AMP)
            #if y_pred:
            #    input_class = 'AMP'
            #else:
            #    input_class = 'non-AMP'

            # Escribir el identificador, secuencia, predicción y probabilidad en el archivo de salida
           # out_f.write(f"{identifier}\t{input_seq}\t{input_class}\t{prob:.4f}\n")
           # print(f"Processed {identifier}: {input_class} (Probability: {prob:.4f})")

# Ruta del archivo FASTA de entrada y del archivo de salida
fasta_file = '/content/drive/MyDrive/data/non_amp_verificacion.fasta'  # Nombre del archivo FASTA con secuencias...
output_file = '/content/drive/MyDrive/Datos Finales para Memoria/resultados/RESULTADO_verificacion_non-AMPs.tsv'  # Archivo de salida con predicciones y probabilidades

# Leer las secuencias del archivo FASTA
sequences = read_fasta(fasta_file)

# Realizar predicciones y escribir resultados en un archivo
predict_amp(sequences, output_file)


Processed non_AMPEP82130: Neuropeptide (Probabilities: 0.0058, 0.0092, 0.9849)
Processed non_AMPEP54142: Neuropeptide (Probabilities: 0.0059, 0.0090, 0.9852)
Processed non_AMPEP137679: Neuropeptide (Probabilities: 0.0059, 0.0090, 0.9852)
Processed non_AMPEP77131: Neuropeptide (Probabilities: 0.0058, 0.0090, 0.9852)
Processed non_AMPEP100173: Neuropeptide (Probabilities: 0.0058, 0.0090, 0.9852)
Processed non_AMPEP49224: Neuropeptide (Probabilities: 0.0059, 0.0090, 0.9852)
Processed non_AMPEP9780: Neuropeptide (Probabilities: 0.0064, 0.0086, 0.9850)
Processed non_AMPEP100390: Neuropeptide (Probabilities: 0.0064, 0.0086, 0.9851)
Processed non_AMPEP119666: Neuropeptide (Probabilities: 0.0059, 0.0090, 0.9850)
Processed non_AMPEP2676: Neuropeptide (Probabilities: 0.0059, 0.0089, 0.9852)
Processed non_AMPEP1965: Neuropeptide (Probabilities: 0.0059, 0.0089, 0.9852)
Processed non_AMPEP133341: Neuropeptide (Probabilities: 0.0059, 0.0090, 0.9851)
Processed non_AMPEP144088: Neuropeptide (Probabili

## TEST DE PREDICCIÓN DE NEUROPEPTIDOS EN BACTERIOCINAS

In [ ]:
import re
import torch

# Función para leer un archivo FASTA
def read_fasta(file_path):
    sequences = []
    with open(file_path, 'r') as f:
        identifier = ""
        sequence = ""
        for line in f:
            line = line.strip()
            if line.startswith(">"):  # Es un identificador de secuencia
                if sequence:  # Guarda la secuencia anterior si existe
                    sequences.append((identifier, sequence))
                    sequence = ""
                identifier = line[1:]  # Guarda el identificador sin el ">"
            else:
                sequence += line  # Agrega las líneas de la secuencia
        if sequence:  # Guarda la última secuencia
            sequences.append((identifier, sequence))
    return sequences

# Función para hacer predicciones de AMP/non-AMP para múltiples secuencias y guardar probabilidades
def predict_amp(sequences, output_file):
    with open(output_file, 'w') as out_f:
        out_f.write("Identifier\tSequence\tPrediction\tProbability\n")  # Encabezados
        for identifier, input_seq in sequences:
            input_seq_spaced = ' '.join([input_seq[i:i+1] for i in range(0, len(input_seq), 1)])
            input_seq_spaced = re.sub(r'[UZOB]', 'X', input_seq_spaced)  # Reemplazar aminoácidos no estándar
            input_seq_tok = tokenizer(input_seq_spaced, return_tensors='pt')
            #print(input_seq_tok)

            output = model(**input_seq_tok)
            logits = output[0]
           # print(f"Logits for {identifier}: {logits}")

            # Extraer la probabilidad de clase AMP
            #y_prob = torch.sigmoid(logits)[:, 1].detach().numpy()
            #y_pred = y_prob > 0.84  # Predicción binaria con umbral de 0.5
            #prob = y_prob[0]  # Probabilidad real (sin umbral)

            # Aplicar softmax para obtener probabilidades de cada clase
            y_prob = torch.softmax(logits, dim=1).detach().numpy()

            # Obtener la clase con la probabilidad más alta
            y_pred_class = y_prob.argmax(axis=1)[0]
            prob_class1, prob_class2, prob_class3 = y_prob[0]  # Probabilidades para las tres clases
            # Determinar la etiqueta de la predicción según la clase
            if y_pred_class == 0:
                input_class = 'non-AMP/OTHERS'  # Cambia estos nombres según tus etiquetas
            elif y_pred_class == 1:
                input_class = 'AMP'
            else:
                input_class = 'Neuropeptide'

            # Escribir el identificador, secuencia, predicción y probabilidades en el archivo de salida
            out_f.write(f"{identifier}\t{input_seq}\t{input_class}\t{prob_class1:.4f}\t{prob_class2:.4f}\t{prob_class3:.4f}\n")
            print(f"Processed {identifier}: {input_class} (Probabilities: {prob_class1:.4f}, {prob_class2:.4f}, {prob_class3:.4f})")


            # Determinar clase (AMP o non-AMP)
            #if y_pred:
            #    input_class = 'AMP'
            #else:
            #    input_class = 'non-AMP'

            # Escribir el identificador, secuencia, predicción y probabilidad en el archivo de salida
           # out_f.write(f"{identifier}\t{input_seq}\t{input_class}\t{prob:.4f}\n")
           # print(f"Processed {identifier}: {input_class} (Probability: {prob:.4f})")

# Ruta del archivo FASTA de entrada y del archivo de salida
fasta_file = '/content/drive/MyDrive/data/bacteriocin_all_class_and_colicin_and_microcin_protlevel_200.fasta'  # Nombre del archivo FASTA con secuencias
output_file = '/content/drive/MyDrive/Datos Finales para Memoria/resultados/RESULTADO_prediccion_bacteriocinas.tsv'  # Archivo de salida con predicciones y probabilidades

# Leer las secuencias del archivo FASTA
sequences = read_fasta(fasta_file)

# Realizar predicciones y escribir resultados en un archivo
predict_amp(sequences, output_file)


Processed 1.1;Anacyclamide_(AcyE): non-AMP/OTHERS (Probabilities: 0.9920, 0.0034, 0.0046)
Processed 10.1;Bovicin_HJ50: non-AMP/OTHERS (Probabilities: 0.9901, 0.0041, 0.0058)
Processed 100.1;Ruminococcin_A_: non-AMP/OTHERS (Probabilities: 0.9928, 0.0030, 0.0043)
Processed 101.1;Salivaricin_9: non-AMP/OTHERS (Probabilities: 0.8151, 0.1751, 0.0098)
Processed 102.1;Salivaricin_A: non-AMP/OTHERS (Probabilities: 0.9927, 0.0033, 0.0040)
Processed 103.1;Salivaricin_A2: non-AMP/OTHERS (Probabilities: 0.9924, 0.0033, 0.0043)
Processed 104.1;Salivaricin_A3: non-AMP/OTHERS (Probabilities: 0.9931, 0.0034, 0.0035)
Processed 105.1;Salivaricin_A4: non-AMP/OTHERS (Probabilities: 0.9928, 0.0035, 0.0037)
Processed 106.1;Salivaricin_A5: non-AMP/OTHERS (Probabilities: 0.9926, 0.0035, 0.0039)
Processed 107.1;Salivaricin_D: AMP (Probabilities: 0.2185, 0.7641, 0.0174)
Processed 108.1;Salivaricin_G32: non-AMP/OTHERS (Probabilities: 0.9878, 0.0064, 0.0058)
Processed 109.1;SalivaricinA: non-AMP/OTHERS (Probabili

## SELECCIÓN DE LAS SECUENCIAS CON POTENCIAL NEUROPEPTÍDICO POSITIVO


In [ ]:
# Función para leer un archivo de predicciones y filtrar aquellos con etiqueta 'Neuropeptide'
def print_neuropeptides(file_path):
    i=0
    with open(file_path, 'r') as f:
        next(f)  # Saltar la primera línea (encabezado)
        for line in f:
            columns = line.strip().split("\t")  # Dividir las columnas por tabulador
            identifier, sequence, prediction, prob_class1, prob_class2, prob_class3 = columns
            # Imprimir solo aquellas líneas con la etiqueta 'Neuropeptide'
            if prediction == 'Neuropeptide':
                print(f"Identifier: {identifier}, Sequence: {sequence}, Probabilities: {prob_class3}")
                i=i+1
        print(f"\nEL NÚMERO DE SECUENCIAS CON POTENCIAL NEUROPEPTÍDICO ES: {i}")
# Ruta del archivo de resultados con predicciones
result_file = '/content/drive/MyDrive/Datos Finales para Memoria/resultados/RESULTADO_prediccion_bacteriocinas.tsv'  # Archivo de salida generado anteriormente

# Llamar a la función para imprimir los neuropeptidos
print_neuropeptides(result_file)


Identifier: 116.1;SGR_1514_putative_Linaridin, Sequence: MSMSPTPAALRGAGGLSESDPGRALSSLAPVTATPGVVAGVALGVALVNAFAAGYNHCGGNVELPM, Probabilities: 0.7664
Identifier: 13.1;Butyrivibriocin, Sequence: MNKDLNALTNPIDEKELEQILGGGDGVFRTISHECHMNTWMFIFTCCS, Probabilities: 0.3600
Identifier: 133.1;Thiomuracin_A_(Thiomuracin_B)_(Thiomuracin_C)_(Thiomuracin_D)_(Thiomuracin_E)_(Thiomuracin_F)_(Thiomuracin_G)_(Thiomuracin_H)_(Thiomuracin_I), Sequence: MDLSDLPMDVFELADDGVAVESLTAGHGMTEVGASCNCFCYICCSCSSA, Probabilities: 0.5838
Identifier: 139.1;Trunkamide, Sequence: MNKKNILPQLGQPVIRLTAGQLSSQLAELSEEALGGVDASTSIAPFCSYDGVDASTSIAPFCSYDGVDASTSIAPFCSYDD, Probabilities: 0.7242
Identifier: 144.1;Thiocillin_GE2270, Sequence: SCNCVCGFCCSCSP, Probabilities: 0.8699
Identifier: 15.1;Catenulipeptin, Sequence: MTEEMTLLDLQGMEQTETDSWGGSGHGGGGDSGLSVTGCNGHSGISLLCDL, Probabilities: 0.7596
Identifier: 197.1;Planosporicin, Sequence: MGISSPALPQNTADLFQLDLEIGVEQSLASPAITSVSWCTPGCTSEGGGSGCSHCC, Probabilities: 0.6302
Identifier: 203.1;Proc

### TEST DEL HOSPITAL RAMÓN Y CAJAL (not completed yet...)

In [ ]:
import re
import torch

# Función para leer un archivo FASTA
def read_fasta(file_path):
    sequences = []
    with open(file_path, 'r') as f:
        identifier = ""
        sequence = ""
        for line in f:
            line = line.strip()
            if line.startswith(">"):  # Es un identificador de secuencia
                if sequence:  # Guarda la secuencia anterior si existe
                    sequences.append((identifier, sequence))
                    sequence = ""
                identifier = line[1:]  # Guarda el identificador sin el ">"
            else:
                sequence += line  # Agrega las líneas de la secuencia
        if sequence:  # Guarda la última secuencia
            sequences.append((identifier, sequence))
    return sequences

# Función para hacer predicciones de AMP/non-AMP para múltiples secuencias y guardar probabilidades
def predict_amp(sequences, output_file):
    i=0
    a=0
    n=0
    total=0
    with open(output_file, 'w') as out_f:
        out_f.write("Identifier\tSequence\tPrediction\tProbability\n")  # Encabezados
        for identifier, input_seq in sequences:
            input_seq_spaced = ' '.join([input_seq[i:i+1] for i in range(0, len(input_seq), 1)])
            input_seq_spaced = re.sub(r'[UZOB]', 'X', input_seq_spaced)  # Reemplazar aminoácidos no estándar
            input_seq_tok = tokenizer(input_seq_spaced, return_tensors='pt')
            #print(input_seq_tok)

            output = model(**input_seq_tok)
            logits = output[0]
           # print(f"Logits for {identifier}: {logits}")

            # Extraer la probabilidad de clase AMP
            #y_prob = torch.sigmoid(logits)[:, 1].detach().numpy()
            #y_pred = y_prob > 0.84  # Predicción binaria con umbral de 0.5
            #prob = y_prob[0]  # Probabilidad real (sin umbral)

            # Aplicar softmax para obtener probabilidades de cada clase
            y_prob = torch.softmax(logits, dim=1).detach().numpy()

            # Obtener la clase con la probabilidad más alta
            y_pred_class = y_prob.argmax(axis=1)[0]
            prob_class1, prob_class2, prob_class3 = y_prob[0]  # Probabilidades para las tres clases
            # Determinar la etiqueta de la predicción según la clase
            if y_pred_class == 0:
                input_class = 'non-AMP/OTHERS'  # Cambia estos nombres según tus etiquetas
                n=n+1
                total=total+1
            elif y_pred_class == 1:
                input_class = 'AMP'
                a=a+1
                total=total+1
            else:
                input_class = 'Neuropeptide'
                i=i+1
                total=total+1

            # Escribir el identificador, secuencia, predicción y probabilidades en el archivo de salida
            out_f.write(f"{identifier}\t{input_seq}\t{input_class}\t{prob_class1:.4f}\t{prob_class2:.4f}\t{prob_class3:.4f}\n")
            print(f"Processed {identifier}: {input_class} (Probabilities: {prob_class1:.4f}, {prob_class2:.4f}, {prob_class3:.4f})")
    print(f"El número de secuencias de AMPs es: {a}")
    print(f"El número de secuencias de non-AMP es: {n}")
    print(f"El número de secuencias de neuropéptidos es: {i}")
    print(f"El número total de secuencias es: {total}")


            # Determinar clase (AMP o non-AMP)
            #if y_pred:
            #    input_class = 'AMP'
            #else:
            #    input_class = 'non-AMP'

            # Escribir el identificador, secuencia, predicción y probabilidad en el archivo de salida
           # out_f.write(f"{identifier}\t{input_seq}\t{input_class}\t{prob:.4f}\n")
           # print(f"Processed {identifier}: {input_class} (Probability: {prob:.4f})")

# Ruta del archivo FASTA de entrada y del archivo de salida
fasta_file = '/content/drive/MyDrive/data/parsed_combined_peptide.fasta'  # Nombre del archivo FASTA con secuencias
output_file = '/content/drive/MyDrive/Datos Finales para Memoria/resultados/RESULTADO_RyC.tsv'  # Archivo de salida con predicciones y probabilidades

# Leer las secuencias del archivo FASTA
sequences = read_fasta(fasta_file)

# Realizar predicciones y escribir resultados en un archivo
predict_amp(sequences, output_file)


Se han truncado las últimas 5000 líneas del flujo de salida.
Processed MH0424_GL0140559: non-AMP/OTHERS (Probabilities: 0.9834, 0.0092, 0.0074)
Processed 890402.BBMN68_1789: non-AMP/OTHERS (Probabilities: 0.9259, 0.0116, 0.0625)
Processed MH0345_GL0067062: Neuropeptide (Probabilities: 0.0059, 0.0089, 0.9852)
Processed MH0074_GL0044846: non-AMP/OTHERS (Probabilities: 0.9921, 0.0039, 0.0040)
Processed MH0086_GL0042171: Neuropeptide (Probabilities: 0.0513, 0.0215, 0.9272)
Processed MH0086_GL0019990: Neuropeptide (Probabilities: 0.0066, 0.0088, 0.9846)
Processed MH0131_GL0028902: non-AMP/OTHERS (Probabilities: 0.9045, 0.0391, 0.0563)
Processed MH0104_GL0084786: non-AMP/OTHERS (Probabilities: 0.9774, 0.0049, 0.0177)
Processed N013A_GL0001357: non-AMP/OTHERS (Probabilities: 0.9814, 0.0117, 0.0068)
Processed MH0153_GL0063928: non-AMP/OTHERS (Probabilities: 0.9812, 0.0146, 0.0042)
Processed MH0184_GL0000442: non-AMP/OTHERS (Probabilities: 0.9914, 0.0043, 0.0042)
Processed MH0003_GL0056148: AMP

KeyboardInterrupt: 